<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 4

## Using [pandas](http://pandas.pydata.org/) and [xarray](http://xarray.pydata.org/en/stable/)  for analysis

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

Let's find a nice time series to examine.

David Robinson's Rutgers Northern Hemisphere Snow Cover Extent is a coarse (88 x 88) northern-hemisphere grid, with data going back to 1966.

http://climate.rutgers.edu/snowcover/docs.php?target=datareq

_Robinson, David A., Estilow, Thomas W., and NOAA CDR Program (2012):NOAA Climate Date Record (CDR) of Northern Hemisphere (NH) Snow Cover Extent (SCE), Version 1. [indicate subset used]. NOAA National Climatic Data Center. doi:10.7289/V5N014G9 [access date]._

Using the URL above, we can find the OpenDAP (DODS) endpoint and access it via the netCDF4 python package:

In [ ]:
import netCDF4
snowcover_url = 'http://www.ncdc.noaa.gov/thredds/dodsC/cdr/snowcover/nhsce_v01r01_19661004_latest.nc'

Open and connect the OpenDAP endpoint:

In [ ]:
ds = netCDF4.Dataset(snowcover_url)

Examine the dataset's title:

In [ ]:
ds.title

And its NetCDF attributes:

In [ ]:
ds.ncattrs()

Find the name of this dataset's Climate Data Record variable:

In [ ]:
ds.cdr_variable

List the dataset's variables:

In [ ]:
ds.variables.keys()

Let's attatch some Python variables to some of the dataset's metadata variables:

In [ ]:
latitude = ds.variables['latitude']
longitude = ds.variables['longitude']
land = ds.variables['land']
area = ds.variables['area']

In [ ]:
print(latitude)

So we see it's an 88 x 88 grid of floats.

We know it's northern-hemisphere data, but what does the grid really look like?

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (10,10)}):
    plt.imshow(land[:], cmap='Accent', interpolation='nearest')

If you are accustomed to looking at polar projections (and if you squint), you can probably see North America on the lower part of the grid.

But we can use the Basemap package from matplotlib to add some grid and coast lines.

In [ ]:
from mpl_toolkits.basemap import Basemap
from ipywidgets import interact
import ipywidgets as widgets

@interact(longitude_0=widgets.IntSlider(min=-165,max=-15,step=30,value=-105))
def plot_land(longitude_0=-80):
    plt.figure(figsize=(10, 10))
    m = Basemap(projection='npstere', boundinglat=30, lon_0=longitude_0)
    m.drawcoastlines()

    parallels = np.arange(0, 90, 20)
    m.drawparallels(parallels, labels=[True])
    meridians = np.arange(-180, 180, 45)
    m.drawmeridians(meridians, labels=[True, True,True,True,True])

    m.pcolor(longitude[:], latitude[:], land[:], latlon=True, cmap='Accent')
    plt.draw()

Let's attatch to the dataset's main CDR variable and get an idea of its contents:

In [ ]:
snowcover = ds.variables['snow_cover_extent']

In [ ]:
print(snowcover)

So, our `snowcover` variable comprises 2574 88 x 88 grids of {0,1} values where `1 = snow_covered` and `0 = no_snow`.

Now we copy all the data from the URL into a local Python variable:

In [ ]:
all_data = snowcover[:,:,:]

We can just plot the data and take a look at a few of the grids:

In [ ]:
@interact(index=widgets.IntSlider(min=0,max=2573,step=4,value=0))
def show_it(index=0):
    with mpl.rc_context(rc={"figure.figsize": (10, 10)}):
        plt.imshow(all_data[index,:,:], interpolation='nearest', cmap='Blues')

So, we have a series of snow / no-snow binary grids.

Previously, we also extracted an `area` grid from our dataset. Areas on 2D maps do not necessarily represent, in all locations, equal areas on the represented 3D surface (i.e. the Earth), and the `area` grid tells us how many physical km<sup>2</sup> each of our grid's cells represents. Let's visualize this information as a colormap:

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (10,10)}):
    plt.imshow(area, interpolation='nearest', cmap="plasma")
    cb = plt.colorbar()
    cb.set_label('Grid Cell Area: $km^2$')

Now we can use NumPy multiplication to scale the snow-covered cells by their corresponding areas and visualize the result:

In [ ]:
@interact(index=widgets.IntSlider(min=0,max=2573,step=4,value=0))
def show_it(index = 0):
    with mpl.rc_context(rc={'figure.figsize': (10,10)}):
        plt.imshow(all_data[index,:,:] * area[:], interpolation='nearest', cmap='plasma')

Let's define a quick routine to compute the total snow-covered area for a grid:

In [ ]:
def snowcover_area_km2(grid, area):
    return np.sum(grid * area)

And compute each weekly total snow-covered area in km<sup>2</sup>:

In [ ]:
weeks = all_data.shape[0]
grid_area = area[:]
total_area = np.ma.zeros(weeks)
for i in np.arange(weeks):
    total_area[i] = snowcover_area_km2(all_data[i, :, :], grid_area)

We'd like to work analytically, not just visually, with these time-dependent data. To do so, we'll need to get our hands on time data matchign the snow-cover data.

Our dataset does provide a `time` variable:

In [ ]:
ds.variables['time']

Let's extract the `time` variable, then use the NetCDF `num2date` helper function to convert `days since ...` into Python `datetime` objects:

In [ ]:
file_time = ds.variables['time']
times = netCDF4.num2date(file_time[:], file_time.units)

Compare the original, basic integer values to the feature-rich `datetime` objects we got:

In [ ]:
print(file_time[3:7])
print(times[3:7])

With corresponding area and time data, we can now plot snow-covered area vs time:

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    plt.plot(times, total_area)
    plt.title('NH Weekly Snow-Covered Area')

This looks like sensible seasonally-oscillating snow-covered area in the northern hemisphere.

We can also look at just a subset of the whole time series:

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    plt.plot(times[100:120], total_area[100:120], marker='.')
    plt.title('Subset of Weekly NH Snow-Covered Area')

If we pick a random point in our `times` variable and compare it to its neighbor, we see that our data has been provided at 7-day resolution:

In [ ]:
offset = 500
times[offset+1] - times[offset]

Pandas provides lots of routines for working with time-series data. In particular, we have [pandas.Series](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html):

_[Series](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#series) is a one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). The axis labels are collectively referred to as the index. The basic method to create a Series is to call:_

      s = pd.Series(data, index=index)

Let's create a `Series` object from our snow-cover and associated time data:

In [ ]:
import pandas as pd

In [ ]:
ts = pd.Series(total_area, index=times)

The `Series.head()` function lets you examine the first few items from a series:

In [ ]:
ts.head()

The `Series.describe()` method give you a statistical overview of your series.

In [ ]:
ts.describe()

`Series` has a built-in `plot` method that will give us something like our original data, plotting the data values vs the index:

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    ts.plot(title='NH Snow-Covered Area: $km^2$')

Examine the index of our `Series`:

In [ ]:
print(ts.index)

So, our `Series`' index is a pandas `DatetimeIndex`. The [documentation for `DatetimeIndex`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html) is pretty overwhelming, so let's just look at a few things one can do with one:

First, let's create a little subindex to play with, choosing 20 values that happen to cross a year boundary:

In [ ]:
subindex = ts.index[50:70]

In [ ]:
print(subindex)

We can access built-in attributes that know about years, months, etc.:

In [ ]:
subindex.year

In [ ]:
subindex.month

In [ ]:
subindex.day

In [ ]:
subindex.dayofyear

We can select data based on the index. Here, again looking at the full time series, we select, and then plot, data that fell in the month of December during any year:

In [ ]:
ts[ts.index.month == 12].head(10)

In [ ]:
ts[ts.index.month == 12].plot(style='.', title='NH December Snow-Covered Area', ylim=(0, 6e7))

Select and then plot the data just from year 2000:

In [ ]:
ts[ts.index.year == 2000].head()

In [ ]:
ts[ts.index.year == 2000].plot(title='NH Snow-Covered Area: 2000')

What was the maximum value across the _entire_ time series?

In [ ]:
ts.max()

On what day did that maxium occur?

In [ ]:
ts[ts == ts.max()]

How often (and when) did snow-covered area reach at least 95% of its all-time maximum? That is, to what extent was the all-time maximum an outlier?

In [ ]:
ts[ts >= ts.max() * .95]

## Resampling time series 

Let's say we need to compare our total snow-covered area with some other geophysical constant, one that is expressed as monthly values. We'll need to transform our weekly data into monthly data.

We can use the `Series.resample()` method to "align" our data with months.

In [ ]:
ts.resample('MS').mean().head()

`'MS'` in the above statement means **M**onthly at **S**tart of month -- without the `S`, the dates would align to the last day of each month.

Let's plot our resampled time series, using the mean value for each month as the plotted value:

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    ts.resample('MS').mean().plot(title='NH Monthly Average Snow-Covered Area')

We can zoom in on a somewhat arbitrary 50-week subset of the resampled time series, comparing it (in blue), with the original weekly data (in green):

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    ts[200:250].resample('MS').mean().plot(marker='x')
    ts[200:250].plot(marker='.')
    plt.title('50 Weeks of NH Snow-Covered Area')

Or, we can look at 50 consecutive months of resampled data:

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    ts.resample('MS').mean()[100:150].plot(title='50 Months of Snow-Covered Area')

We can explicitly show what happens when we call `resample` to take the mean of all values that fell into a month.

Let's look at a run of values in 1966, near the beginning of the time series, that contains the month of November:

In [ ]:
ts.iloc[3:9]

Here's the mean of the November 1966 values:

In [ ]:
print(ts.iloc[4:8])
print("Mean = ", ts.iloc[4:8].mean())

And the head of the resampled time series, which contains the November value:

In [ ]:
ts.resample('MS').mean().head(3)

Note that the '1966-11-01' value for the resampled series matches that explicitly computed from the original time-series values.

But we can improve our analysis.

In this dataset, each indexed snow-cover grid represents a week of data. So the grid with index '1966-10-31' represents the _week_ beginning on that day _through_ 1966-11-06, and some (most!) of the days in that week actually fall in November and should be used to compute November's mean.

Let's resample our time series again, this time using pandas' `ffill()` feature to **f**orward **fill** the missing dates between each week-start date with the provided value. For example, `ffill()` will create the missing dates 1996-11-01 through 1966-11-06 with the value provided for the week starting 1996-10-31.

Here's how pandas sees the original time-series data if we ask for a **D**daily resampling. Note the missing dates between 10-31 and 11-07:

In [ ]:
ts.resample('D').mean()[20:29]

Now using `ffill()`:

In [ ]:
ts.resample('D').ffill(limit=7)[20:29]

So, we succeeding in filling in the missing days with a value copied from the preceding week-start value.

Now when we resample to monthly, we've correctly weighted all of the data for a particular month:

In [ ]:
ts.resample('D').ffill(limit=7).resample('MS').mean().head()

Compare this to the values produced by the more naive resampling we did earlier:

In [ ]:
ts.resample('MS').mean().head()

So far we've been throwing away the results of our resampling, but now let's save our best effort to a variable so we can work more with it:

In [ ]:
monthly = ts.resample('D').ffill(limit=7).resample('MS').mean()

Let's plot values, over the entire time series, for a couple of semi-randomly-selected months:

In [ ]:
with mpl.rc_context(rc={'figure.figsize': (15,3)}):
    monthly[monthly.index.month == 2].plot(linestyle='-', label='February', legend=True)
    monthly[monthly.index.month == 5].plot(marker='.', label='May', legend=True, title='Compare months?')

No surprises there!

We can answer the question: "Which February had the lowest snow-covered area?"

In [ ]:
monthly[monthly.index.month == 2].idxmin()

Or: "What are the rankings of March snow-covered area from greatest to least?"

In [ ]:
march = monthly[monthly.index.month ==  3]
rank = march.rank(ascending=False)
rank.head()

The ranks are given in the second column, and the March with the highest snow-covered area would be the one with rank 1.

In [ ]:
rank[rank == 1.]

To determine the actual snow-covered area for that March, we can query our `march` time series using the index we now know:

In [ ]:
march[march.index == '1978-03-01']

Now let's take a look at [`pandas.DataFrame`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)  

     Two-dimensional size-mutable, potentially heterogeneous tabular data
     structure with labeled axes (rows and columns). Arithmetic operations
     align on both row and column labels. Can be thought of as a dict-like
     container for Series objects. The primary pandas data structure.

We can use a `DataFrame` to align March rank and values into a single object.

Let's create a simple `DataFrame` from our monthly data using March and its associated ranks. The `march` series contains the actual snow-cover values, and the `rank` series contains the rankings but, if we look at the heads of both series, we see that the indexes are the same:

In [ ]:
march.head()

In [ ]:
rank.head()

This makes it trivial to create a `DataFrame` comprising both series. We give the `DataFrame` constructor the series we want to use along with labels for those series:

In [ ]:
df = pd.DataFrame(data={'march': march, 'rank': rank})

In [ ]:
df.head()

The columns of the `DataFrame` have the names we specified (conveniently, in this case, the same as the names of the underlying series, though this is not a requirement):

In [ ]:
df.columns

And we can use the column names to refer to the series and, for example, sort the data based on the "rank" column:

In [ ]:
df.sort_values('rank', ascending=True).head()

It's nice to be able to use named columns instead of trying to remember numeric indexes as one might in, for example, pure NumPy. Here we get the same result, in a single statement, as we obtained previously by making separate queries on the original snow-cover and rank series.

Say that we now want to compute the anomalies from the mean for all Marches across the time series. With a `DataFrame`, we can simply add a new column representing the difference between the all-Marches mean from each value:

In [ ]:
df['march_anomaly'] = (df['march'] - df['march'].mean())
df.head()

You can add columns at will to a pandas `DataFrame`.

Now that we've seen how a basic `DataFrame` works, let's create a new one from the Northern Hemisphere `monthly` snow-covered area series.

A quick review of the series in question:

In [ ]:
monthly.head()

Create a `DataFrame` using the `Series`:

In [ ]:
df2 = pd.DataFrame(monthly, columns=['snowcover'])
df2.head()

This doesn't look so different from the series, but now that we're working with a `DataFrame`, we can columns to the index, creating a so-called "multi-index":

In [ ]:
df2 = df2.set_index([df2.index.year, df2.index.month])
df2.head()

In [ ]:
type(df2.index)

Now, our index has two levels, which we can examine independently:

In [ ]:
print(df2.index.levels[0])
print(df2.index.levels[1])

So, our level-0 index specifies years, and our level-1 index specifies months.

We can select data from this index using the [`loc()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html) method on `DataFrame`:

In [ ]:
df2.loc[1979:1980]

Or we can use a cross section (the [`xs()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.xs.html) method on `DataFrame`) to grab a specific month (May=5) and display the results by year:

In [ ]:
df2.xs(5, level=1).loc[1980:1985]

We can use [`unstack()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) on the months' levels to get an index of years, with columns of months. This pulls the specificed index (level 1 is month) out and spreads it as columns under the existing _snowcover_ column index:

In [ ]:
year_by_month = df2.unstack(level=1)
year_by_month.head()

If we prefer rows of months and columns of years, we could unstack level 0:

In [ ]:
month_by_year = df2.unstack(level=0)
month_by_year.head(3)

We can still select from the `DataFrame`, first selecting the _snowcover_ column (which contains all the years), and then subsetting by year...

In [ ]:
month_by_year['snowcover'][[1970, 1980, 1990, 2000, 2010 ]]

...and plotting these data:

In [ ]:
month_by_year['snowcover'][[1970, 1980, 1990, 2000, 2010 ]].plot()

We can save our work by writing the monthly data out to a CSV file:

In [ ]:
monthly.name = 'snowcover'
monthly.to_csv('monthly-extents.csv', index_label='date', header=True)

In [ ]:
%%bash
ls monthly-extents.csv

In [ ]:
%%bash
head monthly-extents.csv